<a href="https://colab.research.google.com/github/waynezmc/DL-ML/blob/master/Emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

## Import packages

In [0]:
import tensorflow as tf
import os
import zipfile
from tensorflow import keras

## Download the datasets and unzip them

In [52]:
DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2019-03-13 11:19:00--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-03-13 11:19:00 (132 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



## Define the callback functions

In [0]:
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc') > DESIRED_ACCURACY):
            print("\nReached 99.9% accuracy so cancelling trainning!");
            self.model.stop_training = True

callbacks = myCallback()

## Define and compile the model

In [0]:
model = keras.models.Sequential([
    keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPool2D(2, 2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPool2D(2, 2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPool2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

In [0]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])
#model.compile(loss='binary_crossentropy',
              #optimizer=RMSprop(lr=0.001),
              #metrics=['acc'])

## Create ImageDataGenerator

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)

In [57]:
train_generator = train_datagen.flow_from_directory(
    "/tmp/h-or-s/",
    target_size = (150,150),
    batch_size = 10,
    class_mode = 'binary'
)

Found 80 images belonging to 2 classes.


## Start training

In [58]:
history = model.fit_generator(
    train_generator,
    #steps_per_epoch=8,
    epochs=15,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/15
8/8 [==============================] - 0s 47ms/step - loss: 2.7517 - acc: 0.4875
Epoch 2/15
8/8 [==============================] - 0s 46ms/step - loss: 0.7062 - acc: 0.6625
Epoch 3/15
8/8 [==============================] - 0s 26ms/step - loss: 0.4405 - acc: 0.7750
Epoch 4/15
8/8 [==============================] - 0s 45ms/step - loss: 0.2018 - acc: 0.8875
Epoch 5/15
8/8 [==============================] - 0s 27ms/step - loss: 0.1139 - acc: 0.9625
Epoch 6/15
8/8 [==============================] - 0s 45ms/step - loss: 0.0777 - acc: 0.9500
Epoch 7/15
8/8 [==============================] - 0s 27ms/step - loss: 0.0523 - acc: 0.9875
Epoch 8/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0107 - acc: 1.0000
Reached 99.9% accuracy so cancelling trainning!
8/8 [==============================] - 0s 35ms/step - loss: 0.0096 - acc: 1.0000


In [0]:
keras.backend.clear_session()

In [59]:
print(steps_per_epoch)

NameError: ignored